In [155]:
from scipy.integrate import quad
from collections import deque
from dataclasses import dataclass
from typing import NamedTuple, Callable

class Position(NamedTuple):
    owner: str
    X_start: float
    displacement: float
    price: float

    # round to 2 decimal places
    def __repr__(self) -> str:
        return f"Position(owner={self.owner}, X_start={round(self.X_start, 2)}, displacement={round(self.displacement, 2)}, price={round(self.price, 2)})"




# cost of buying d portion of the asset starting at X_0
def price(X_0, d, marginal_price):
    ans, _ = quad(marginal_price, X_0, X_0 + d)
    return ans

class AuctionAmm:
    def __init__(self, marginal_price, X_init):
        self.marginal_price: Callable[[float], float] = marginal_price
        self.X = X_init
        self.owners: deque[Position] = deque()
        self.balance = 0

    def print(self) -> str:
        # print self.X, self.balance, and self.owners line by line
        print(f"AuctionAmm(X={self.X}, balance={self.balance}")
        for position in self.owners:
            print(position)

        # return f"AuctionAmm(X={self.X}, balance={self.balance}, owners={self.owners})"
    

    # integral of price function from self.X to self.X + d
    def buy(self, owner, d):
        assert 0 < d <= 1, "d must be in (0, 1]"
        p = price(self.X, d, self.marginal_price)
        self.balance += p
        
        window_start = self.X + d - 1
        while self.owners:
            first_owner, first_X_0, first_d, first_p = self.owners[0]
            first_X_end = first_X_0 + first_d
            if first_X_0 > window_start:
                break
            if first_X_end <= window_start:
                self.owners.popleft()
                self.balance -= price(first_X_0, first_d, self.marginal_price)
                continue
            # X_0 < window_start < X_end
            new_d = first_X_end - window_start
            new_price = price(window_start, new_d, self.marginal_price)
            refund = first_p - new_price
            self.balance -= refund
            self.owners[0] = Position(first_owner, window_start, new_d, new_price)   
            break 

        
        if self.owners:
            last_owner, last_X_0, last_d, last_p = self.owners[-1]
            if last_owner == owner:
                new_pos = Position(owner, last_X_0, d + last_d, p + last_p)
                assert abs(new_pos.X_start + new_pos.displacement - (self.X + d)) < 1e-2, f"error in displacement calculation: {new_pos.X_start} + {new_pos.displacement} != {self.X + d}"
                price_calculated = price(new_pos.X_start, new_pos.displacement, self.marginal_price)
                assert abs(new_pos.price - price_calculated) < 1e-2, f"error in price calculation: {new_pos.price} != {price_calculated}"
                self.owners[-1] = new_pos
                self.X += d
                return new_pos
        new_pos = Position(owner, self.X, d, p)
        self.owners.append(new_pos)
        self.X += d

        return new_pos
        

In [156]:
amm = AuctionAmm(
    lambda x: x, 1
)

amm

In [157]:
amm.buy("dice", 0.5)
amm.buy("dice", 0.5)
amm.buy("dice", 0.5)
amm.buy("devo", 0.3)

Position(owner=devo, X_start=2.5, displacement=0.3, price=0.79)

In [158]:
amm.print()
amm.buy("devo", 0.3)
amm.print()

AuctionAmm(X=2.8, balance=2.3
Position(owner=dice, X_start=1.8, displacement=0.7, price=1.51)
Position(owner=devo, X_start=2.5, displacement=0.3, price=0.79)
AuctionAmm(X=3.0999999999999996, balance=2.6
Position(owner=dice, X_start=2.1, displacement=0.4, price=0.92)
Position(owner=devo, X_start=2.5, displacement=0.6, price=1.68)


In [159]:
amm2 = AuctionAmm(
    lambda x: 2**x, 1
)

In [161]:
amm2.buy("dice", 1)
amm2.print()

AuctionAmm(X=3.0999999999999996, balance=2.6
Position(owner=dice, X_start=2.1, displacement=0.4, price=0.92)
Position(owner=devo, X_start=2.5, displacement=0.6, price=1.68)


In [163]:
amm2.print()
amm2.buy("devo", 0.5)
amm2.print()

AuctionAmm(X=3, balance=5.770780163555853
Position(owner=dice, X_start=2, displacement=1, price=5.77)
AuctionAmm(X=3.5, balance=8.161115572774316
Position(owner=dice, X_start=2.5, displacement=0.5, price=3.38)
Position(owner=devo, X_start=3, displacement=0.5, price=4.78)


In [165]:
amm2.buy("drew", 0.1)

Position(owner=drew, X_start=3.5, displacement=0.1, price=1.17)

In [166]:
amm2.print()

AuctionAmm(X=3.6, balance=8.746867095591194
Position(owner=dice, X_start=2.6, displacement=0.4, price=2.79)
Position(owner=devo, X_start=3, displacement=0.5, price=4.78)
Position(owner=drew, X_start=3.5, displacement=0.1, price=1.17)


In [167]:
amm2.buy("axel", 1)

Position(owner=axel, X_start=3.6, displacement=1, price=17.49)

In [168]:
amm2.print()

AuctionAmm(X=4.6, balance=17.493734191182373
Position(owner=drew, X_start=3.6, displacement=0.0, price=0.0)
Position(owner=axel, X_start=3.6, displacement=1, price=17.49)


In [169]:
a = AuctionAmm(

    lambda x: x, 1
)

In [170]:
a.buy("pinky", 1)

Position(owner=pinky, X_start=1, displacement=1, price=1.5)

In [171]:
a.print()

AuctionAmm(X=2, balance=1.5
Position(owner=pinky, X_start=1, displacement=1, price=1.5)


In [172]:
a.buy("dice", 0.5)

Position(owner=dice, X_start=2, displacement=0.5, price=1.12)

In [173]:
a.print()

AuctionAmm(X=2.5, balance=2.0
Position(owner=pinky, X_start=1.5, displacement=0.5, price=0.88)
Position(owner=dice, X_start=2, displacement=0.5, price=1.12)


In [174]:
a.buy("larry", 0.1)

Position(owner=larry, X_start=2.5, displacement=0.1, price=0.26)

In [175]:
a.print()

AuctionAmm(X=2.6, balance=2.1
Position(owner=pinky, X_start=1.6, displacement=0.4, price=0.72)
Position(owner=dice, X_start=2, displacement=0.5, price=1.12)
Position(owner=larry, X_start=2.5, displacement=0.1, price=0.26)
